## Install Requirements

In [0]:
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal

## Mount gDrive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

## Folder Paths

In [0]:
# IMAGE_FOLDER_ID = "1dNrhrHBgfBgCcNzzrn-Kf6xAKxrPhnwl"
# TEMPT_FOLDER_ID = "1bGHeob3mTuPVswoXfaSph9OKhf2cLJG8"
# MASKS_FOLDER_ID = "1bWeAOvdLI-e_8l6fENUibImTnRbVsGB5"

HOME_DIR_PATH = "/content/drive/My Drive/"

IMAGE_FOLDER_PATH = HOME_DIR_PATH + "CropYield/MOD09A1_2003_2018/"
TEMPT_FOLDER_PATH = HOME_DIR_PATH + "CropYield/MYD11A2_2003_2018/"
MASKS_FOLDER_PATH = HOME_DIR_PATH + "CropYield/MCD12Q1_2003_2018/"

SOYBEANS_DATA_PATH = HOME_DIR_PATH + "CropYield/NASS_Soybeans_2003_2018.csv"

SAVE_DIR_PATH = HOME_DIR_PATH + "CropYield/Processed_Images/"

In [0]:
from os import listdir
files_names = listdir(IMAGE_FOLDER_PATH)

## Process Images

In [0]:
from pathlib import Path

import numpy  as np
import pandas as pd

import gdal
import math
from itertools import repeat
from concurrent.futures import ProcessPoolExecutor

class ProcessImages:
    """
    Take the exported, downloaded data and clean it.

    Specifically,
    - split the image collections into years
    - merge the bands from temperature and images into one file
    - apply the mask so that only cropland is considered

    Parameters
    -----------
    image_path: 
        path to which the image tif files have been saved
    tempt_path: 
        path to which the temperature tif files have been saved
    masks_path: 
        path to which the mask tif files have been saved
    yield_data: 
        path to the yield data csv file
    save_dir: 
        path to save the data to
    multiprocessing: 
        boolean for whether to use multiprocessing
    """
    def __init__(self, image_path, tempt_path, masks_path, yield_path, save_dir, 
                 multiprocessing=True):
        
        self.image_path = Path(image_path)
        self.tempt_path = Path(tempt_path)
        self.masks_path = Path(masks_path)
        
        self.yield_path = Path(yield_path)
        self.save_dir   = Path(save_dir)
        
        self.tif_files  = listdir(image_path)[:]

        self.multiprocessing = False
        self.processes   = 4
        self.parallelism = 6

        self.save_dir   = Path(save_dir)
        if not self.save_dir.exists():
            self.save_dir.mkdir()

        self.yield_data = get_yield_data(yield_path)
        

    def process(self, num_years=16, delete_when_done=False):
        """
        Process all the data.

        Parameters
        ----------
        num_years:
            integer years of data to create.
        delete_when_done:
            boolean for whether or not to delete the original .tif files
        """
        if delete_when_done: print("Warning! .tif files will be deleted.")
            
        # Process images one-by-one.    
        if not self.multiprocessing:
            for filename in self.tif_files:
                process_county(
                    filename, self.save_dir, 
                    self.image_path, self.masks_path, self.tempt_path, self.yield_data, 
                    num_years=num_years, delete_when_done=delete_when_done
                )
        
        # Parallel process images.
        else:
            
            length     = len(self.tif_files)
            files_iter = iter(self.tif_files)

            # Turn all other arguments to iterators.
            
            savedir_iter   = repeat(self.save_dir)
            im_path_iter   = repeat(self.image_path)
            mask_path_iter = repeat(self.massk_path)
            temp_path_iter = repeat(self.tempt_path)
            
            yield_iter     = repeat(self.yield_data)
            num_years_iter = repeat(num_years)
            
            delete_when_done_iter = repeat(delete_when_done)

            with ProcessPoolExecutor() as executor:
                chunksize = int(max(length / (self.processes * self.parallelism), 1))
                executor.map(process_county, 
                    files_iter, savedir_iter, 
                    im_path_iter, mask_path_iter, temp_path_iter, yield_iter, 
                    num_years_iter, delete_when_done_iter, chunksize=chunksize
                )

def get_yield_data(yield_path):
    """
    Cleans the yield data by making sure any Nan values in the columns we 
    care about are removed.
    """
    cols_nan = ['Year', 'State ANSI', 'County ANSI', 'Value']
    yield_values = pd.read_csv(yield_path).dropna(subset=cols_nan, how='any')
    cols_val = ['Year', 'State ANSI', 'County ANSI']
    return yield_values[cols_val].values
                
                
def process_county(filename, savedir, image_path, masks_path, tempt_path, yield_data, num_years,
                   delete_when_done):
    """
    Process and save county level data
    """

    # Files have "{state}_{county}.tif" format, the last 4 characters are ".tif".
    locations = filename[:-4].split("_")
    state, county = int(locations[0]), int(locations[1])

    image = np.transpose(
        np.array(gdal.Open(str(image_path / filename)).ReadAsArray(), dtype='uint16'),
        axes=(1, 2, 0)
    )

    tempt = np.transpose(
        np.array(gdal.Open(str(tempt_path / filename)).ReadAsArray(), dtype='uint16'),
        axes=(1, 2, 0)
    )

    masks = np.transpose(
        np.array(gdal.Open(str(masks_path / filename)).ReadAsArray(), dtype='uint16'),
        axes=(1, 2, 0)
    )
    
    # A value of 12 indicates cropland; everything else, we want to ignore
    masks[masks != 12] = 0
    masks[masks == 12] = 1

    # when exporting the image, we appended bands from many years into a single image for efficiency. We want
    # to split it back up now

    # num bands and composite period from the MODIS website
    image_list = divide_into_years(image, bands=7, composite_period=  8, num_years=num_years)
    tempt_list = divide_into_years(tempt, bands=2, composite_period=  8, num_years=num_years)
    masks_list = divide_into_years(masks, bands=1, composite_period=365, num_years=num_years, extend=True)

    image_tempt_merged = merge_image_lists(image_list, 7, tempt_list, 2)
    masked_image_tempt = mask_image(image_tempt_merged, masks_list)
    
    assert len(masked_image_tempt) == num_years, "Dataset does not align with given number of years."

    # start year from the MODIS website
    start_year = 2003  
    
    for i in range(0, num_years):
        year = i + start_year

        key = np.array([year, state, county])
        
        # Check if this key is in the yield data.
        if np.equal(yield_data[:, :3], key).all(axis=1).max():
            save_filename = f'{year}_{state}_{county}'
            np.save(savedir / save_filename, masked_image_tempt[i])
    
    # Delete .tif files if delete_when_done flag is set.
    if delete_when_done:
        (image_path / filename).unlink()
        (temperature_path / filename).unlink()
        (mask_path / filename).unlink()
        
    print(f'{filename} array written')


# helper methods for the data cleaning class

def divide_into_years(image, bands, composite_period, num_years=16, extend=False):
    """
    Parameters
    ----------
    image: the appended image collection to split up
    bands: the number of bands in an individual image
    composite_period: length of the composite period, in days
    num_years: how many years of data to create.
    extend: boolean, default=False
        If true, and num_years > number of years for which we have data, then the extend the image
        collection by copying over the last image.
        NOTE: This is different from the original code, where the 2nd to last image is copied over

    Returns:
    ----------
    im_list: a list of appended image collections, where each element in the list is a year's worth
        of data
    """
    bands_per_year = bands * math.ceil(365 / composite_period)

    # Pad the images with the last image if flag 'extend' is set.
    if extend:
        num_bands_necessary = bands_per_year * num_years
        while image.shape[2] < num_bands_necessary:
            image = np.concatenate((image, image[:, :, -bands:]), axis=2)

    image_list = []
    cur_idx = 0
    for i in range(0, num_years):
        image_list.append(image[:, :, cur_idx:cur_idx + bands_per_year])
        cur_idx += bands_per_year
        
    # image_list.append(img[:, :, cur_idx:])
    return image_list


def merge_image_lists(im_list_1, num_bands_1, im_list_2, num_bands_2):
    """
    Given two image lists (i.e. the MODIS images and the MODIS temperatures),
    merges them together.

    Parameters
    ----------
    im_list_1: the first image list to merge, where an image list is the output of
        divide_into_years. Note that im_list_1 and im_list_2 must be the same length
        (i.e. the num_years parameter in divide_into_years must be the same when both image
        lists are created)
    num_bands_1: int
        The number of bands in each image in im_list_1
    im_list_2: the second image list to merge, where an image list is the output of
        divide_into_years
    num_bands_2: int

    Returns
    ----------
    merged_im_list: A merged image list
    """
    merged_list = []

    assert len(im_list_1) == len(im_list_2), "Image lists are not the same length!"

    for im1, im2 in zip(im_list_1, im_list_2):
        individual_images = []

        # split the 'year' appended images into individual images
        for image_1, image_2 in zip(np.split(im1, im1.shape[-1] / num_bands_1, axis=-1),
                                    np.split(im2, im2.shape[-1] / num_bands_2, axis=-1)):
            individual_images.append(np.concatenate((image_1, image_2), axis=-1))
        merged_list.append(np.concatenate(individual_images, axis=-1))
    return merged_list


def mask_image(im_list, mask_list):
    masked_im_list = []

    assert len(im_list) == len(mask_list), "Mask and Image lists are not the same length!"

    for img, mask in zip(im_list, mask_list):
        expanded_mask = np.tile(mask, (1, 1, img.shape[2]))
        masked_img = img * expanded_mask
        masked_im_list.append(masked_img)

    return masked_im_list

In [0]:
process = ProcessImages(
    IMAGE_FOLDER_PATH, TEMPT_FOLDER_PATH, MASKS_FOLDER_PATH, 
    SOYBEANS_DATA_PATH, SAVE_DIR_PATH
)

In [0]:
process.process()

1_47.tif array written
1_65.tif array written
1_85.tif array written
1_105.tif array written
1_119.tif array written
1_3.tif array written
1_53.tif array written
1_99.tif array written
1_9.tif array written
1_19.tif array written
1_43.tif array written
1_49.tif array written
1_55.tif array written
1_71.tif array written
1_33.tif array written
1_77.tif array written
1_79.tif array written
1_83.tif array written
1_89.tif array written
1_103.tif array written
1_21.tif array written
1_57.tif array written
1_121.tif array written
1_125.tif array written
1_5.tif array written
1_39.tif array written
1_61.tif array written
1_69.tif array written
1_109.tif array written
5_105.tif array written
5_1.tif array written
5_35.tif array written
5_37.tif array written
5_77.tif array written
5_85.tif array written
5_95.tif array written
5_107.tif array written
5_117.tif array written
5_123.tif array written
5_147.tif array written
5_21.tif array written
5_31.tif array written
5_55.tif array written
5_63

In [0]:
files_names.index('48_215.tif')

997

In [0]:
files_names[998:]

['48_215.tif',
 '1_47.tif',
 '1_65.tif',
 '1_85.tif',
 '1_105.tif',
 '1_119.tif',
 '1_3.tif',
 '1_53.tif',
 '1_99.tif',
 '1_9.tif',
 '1_19.tif',
 '1_43.tif',
 '1_49.tif',
 '1_55.tif',
 '1_71.tif',
 '1_33.tif',
 '1_77.tif',
 '1_79.tif',
 '1_83.tif',
 '1_89.tif',
 '1_103.tif',
 '1_21.tif',
 '1_57.tif',
 '1_121.tif',
 '1_125.tif',
 '1_5.tif',
 '1_39.tif',
 '1_61.tif',
 '1_69.tif',
 '1_109.tif',
 '5_105.tif',
 '5_1.tif',
 '5_35.tif',
 '5_37.tif',
 '5_77.tif',
 '5_85.tif',
 '5_95.tif',
 '5_107.tif',
 '5_117.tif',
 '5_123.tif',
 '5_147.tif',
 '5_21.tif',
 '5_31.tif',
 '5_55.tif',
 '5_63.tif',
 '5_67.tif',
 '5_75.tif',
 '5_93.tif',
 '5_111.tif',
 '5_121.tif',
 '5_145.tif',
 '5_7.tif',
 '5_3.tif',
 '5_17.tif',
 '5_43.tif',
 '5_41.tif',
 '5_69.tif',
 '5_79.tif',
 '10_1.tif',
 '10_3.tif',
 '10_5.tif',
 '13_91.tif',
 '13_23.tif',
 '13_175.tif',
 '13_225.tif',
 '13_33.tif',
 '13_163.tif',
 '13_165.tif',
 '13_147.tif',
 '13_221.tif',
 '13_55.tif',
 '13_129.tif',
 '13_295.tif',
 '13_19.tif',
 '13_27